# Generalise Huckel Solver

> Constructs the Huckel matrix and solves for the values, based on Ex 3.1 instructions

This solver will only work if all $p$ orbitals are equivalent (it will not consider for instance perturbations due to nitrogen in pyridine).
The Huckel theory assumptions are:
$$ H_{ij}= \begin{cases} \alpha,\  i=j\\ \beta,\  i \text{ adjacent to } j\\ 0,\  \text{otherwise}\end{cases} $$
with scaling such that $\alpha = 0$ and $\beta =-1$. As such, it can be seen that the resulting Huckel matrix is the negative of the adjacency matrix $A$ where: 
$$ A_{ij}= \begin{cases}  1,\  i \text{ adjacent to } j\\ 0,\  \text{otherwise}\end{cases} $$

In [ ]:
#| default_exp optimisation

In [ ]:
#| export
#| echo: false
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#| export
def generate_smiles(type, n):
    """
    generate the smiles of either a straight chain or ring polyene, with n atoms.
    All carbons will be sp2 hybridised.
    For linear molecules with an odd number of atoms it will return the anion
    For rings with 4n+1 atoms it will return the anion eg C5H5-    
    For rings with 4n+3 atoms it will return the cation eg C7H7+   
    """
    unit = 'C=C'# the base 2 carbon double bond unit
    if n<3: type = 'linear' #ring needs at least 3 atoms
    if n <=1: 
        raise Exception("More than 1 Carbon atoms is needed")
    if not isinstance(n, int): 
        raise Exception("An integer number of atoms is needed")
    if type == 'linear':
        repeats = n//2 # repeats of the base unit of 2 carbons
        terminal = n%2 # whether or not a terminal carbon is needed
        output = repeats*unit +terminal*"[CH2+]"
    if type == 'ring':
        repeats = n//2 # repeats of the base unit of 2 carbons
        cation_terminal = 1 if n%4==3 else 0 # if 4n+3 carbon atoms: cation eg C7H7+
        anion_terminal = 1 if n%4==1 else 0 # if 4n+3 carbon atoms: cation eg C5H5-
        output = "C1=C" +(repeats-1)*unit +anion_terminal*"[CH-]"+cation_terminal*"[CH+]"+"1"
    return(output)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()